In [1]:
import pandas as pd

In [4]:
data = pd.read_csv("15-16.csv", parse_dates=True, dayfirst=True, index_col=[1])

In [5]:
data[:]

,Div,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA
Date,,,,,,,,,,,,,,,,,,,,,
2008-08-15,E0,Bournemouth,Aston Villa,0,1,A,0,0,D,M Clattenburg,...,2.11,2.02,1.88,1.79,26,-0.50,1.98,1.93,1.99,1.92
2008-08-15,E0,Chelsea,Swansea,2,2,D,2,1,H,M Oliver,...,1.88,1.80,2.07,1.99,27,-1.50,2.24,2.16,1.80,1.73
2008-08-15,E0,Everton,Watford,2,2,D,0,1,A,M Jones,...,1.93,1.84,2.03,1.96,26,-1.00,2.28,2.18,1.76,1.71
2008-08-15,E0,Leicester,Sunderland,4,2,H,3,0,H,L Mason,...,2.27,2.18,1.73,1.67,26,-0.50,2.00,1.95,1.96,1.90
2008-08-15,E0,Man United,Tottenham,1,0,H,1,0,H,J Moss,...,1.86,1.79,2.13,2.01,26,-1.00,2.20,2.09,1.82,1.78
2008-08-15,E0,Norwich,Crystal Palace,1,3,A,0,1,A,S Hooper,...,2.25,2.18,1.75,1.67,27,0.00,1.83,1.78,2.17,2.08
2009-08-15,E0,Arsenal,West Ham,0,2,A,0,1,A,M Atkinson,...,1.71,1.64,2.30,2.22,27,-1.50,1.96,1.89,2.06,1.96
2009-08-15,E0,Newcastle,Southampton,2,2,D,1,1,D,C Pawson,...,2.23,2.14,1.80,1.70,27,0.00,2.06,1.98,1.92,1.87
2009-08-15,E0,Stoke,Liverpool,0,1,A,0,0,D,A Taylor,...,2.15,2.06,1.82,1.75,26,0.25,2.00,1.95,1.96,1.91
